In [21]:
import pandas as pd
import os
import math
import numpy as np

In [32]:
code_to_attribute_dict = {'FL': 'Leaves_fresh', 'ML': 'Leaves_mature', 'DL': 'Leaves_old', 
                          'BD': 'Flowers_bud', 'OF': 'Flowers_open', 'MF': 'Flowers_male', 
                          'FF': 'Flowers_Female', 'UFR': 'Fruits_unripe', 'RFR': 'Fruits_ripe', 
                          'OFR': 'Fruits_open'}
code_to_category_dict = {'NA': None, '': None, '0': 0, '1': 1, '2': 2} #TODO - what should map to -1?

def code_to_attribute_category_tuple(code):
    code_attr_tup = code.split('_')
    code_attr_tup[0] = code_to_attribute_dict[code_attr_tup[0]]
    code_attr_tup[1] = code_to_category_dict[code_attr_tup[1]]
    return tuple(code_attr_tup)

def get_duplicates_df(df):
    duplicates_df = pd.DataFrame({'species_id': [], 'all_same': []})
    for species_id in df['species_id'].unique():
        df_filtered = df[df['species_id'] == species_id]
        if len(df_filtered) > 1:
            # see if categorical vectors are all the same:
            all_equal = True
            for colname in df.columns[3:-2]:
                all_equal = all_equal and df_filtered[colname].nunique() == 1
            duplicates_df.loc[len(duplicates_df) + 1] = pd.Series({'species_id': species_id, 'all_same': all_equal})
    return duplicates_df

def clean_df(df):
    # TODO:
    # - get rid of null values

    # get rid of duplicate rows
    print("getting rid of duplicates")
    duplicates_df = get_duplicates_df(df)
    for i in range(len(duplicates_df)):
        dup_id = duplicates_df.iloc[i]['species_id']
        drop_idx = list(df['species_id']).index(dup_id)
        df = df.drop(index=drop_idx)
    print("got rid of duplicates")
    # new columns: everything in df that is not a month, plus the 10 categorical codes, plus a 'week' column
    week_codes = ['Jan_wk1', 'Jan_wk2', 'Jan_wk3', 'Jan_wk4', 'Feb_wk1', 
                   'Feb_wk2', 'Feb_wk3', 'Feb_wk4', 'Mar_wk1', 'Mar_wk2',
                   'Mar_wk3', 'Mar_wk4', 'Apr_wk1', 'Apr_wk2', 'Apr_wk3', 'Apr_wk4',
                   'May_wk1', 'May_wk2', 'May_wk3', 'May_wk4', 'Jun_wk1', 'Jun_wk2',
                   'Jun_wk3', 'Jun_wk4', 'Jul_wk1', 'Jul_wk2', 'Jul_wk3', 'Jul_wk4',
                   'Aug_wk1', 'Aug_wk2', 'Aug_wk3', 'Aug_wk4', 'Sep_wk1', 'Sep_wk2',
                   'Sep_wk3', 'Sep_wk4', 'Oct_wk1', 'Oct_wk2', 'Oct_wk3', 'Oct_wk4',
                   'Nov_wk1', 'Nov_wk2', 'Nov_wk3', 'Nov_wk4', 'Dec_wk1', 'Dec_wk2','Dec_wk3', 'Dec_wk4']
    
    cat_codes = ['Leaves_fresh', 'Leaves_mature', 'Leaves_old', 'Flowers_bud',
       'Flowers_open', 'Flowers_male', 'Flowers_Female', 'Fruits_unripe',
       'Fruits_ripe', 'Fruits_open']

    base_cols = [not_week_code for not_week_code in filter(lambda c: c not in week_codes, list(df.columns))]
    try:
        base_cols.remove('created_at')
    except:
        print("WARNING: created_at column not found")
    
    new_cols = base_cols + ['week'] + cat_codes
    
    new_df = pd.DataFrame(columns=new_cols)

    num_idxs = len(df)
    for idx, row in df.iterrows():
        if idx % 10 == 0:
            print("{} % done".format(100 * idx / num_idxs))
        for week_idx, week_name in enumerate(week_codes):
            new_datapoint = {}
            for old_colname in base_cols:
                new_datapoint[old_colname] = row[old_colname]
            new_datapoint['week'] = week_idx
            cat_vector_for_week = row[week_name]

            # make nan map to a cat vector of all Nones
            # print(type(cat_vector_for_week), cat_vector_for_week)
            if not isinstance(cat_vector_for_week, str):
                if math.isnan(cat_vector_for_week):
                    for attr_name in code_to_attribute_dict.values():
                        new_datapoint[attr_name] = None
                    new_df.loc[len(new_df)] = new_datapoint
                    continue
            
            cat_vector_list = list(map(code_to_attribute_category_tuple, cat_vector_for_week.split(",")))
            for (attr, cat) in cat_vector_list:
                new_datapoint[attr] = cat

            # add new datapoint to df
            new_df.loc[len(new_df)] = new_datapoint

    return new_df

In [33]:
# clean all old pvt dataframes, put in ./reference_data
new_dir_path = 'reference_data'
try:
    # Create the directory
    os.mkdir(new_dir_path)
    print(f"Directory '{new_dir_path}' created.")
except FileExistsError:
    print(f"Directory '{new_dir_path}' already exists.")

for tablename in os.listdir('./pvttables_raw/'):
    if tablename == '.DS_Store' or tablename in os.listdir('./reference_data/'):
        continue
    print("cleaning {}".format(tablename))
    df = pd.read_csv('./pvttables_raw/{}'.format(tablename), sep=';')
    new_df = clean_df(df)
    new_df.to_csv('./reference_data/{}'.format(tablename))

Directory 'reference_data' already exists.
cleaning pvt_pvt_haryana.csv
getting rid of duplicates
got rid of duplicates
0.0 % done
4.184100418410042 % done
8.368200836820083 % done
12.552301255230125 % done
16.736401673640167 % done
20.92050209205021 % done
25.10460251046025 % done
29.288702928870293 % done
33.47280334728033 % done
37.65690376569037 % done
41.84100418410042 % done
46.02510460251046 % done
50.2092050209205 % done
54.39330543933055 % done
62.76150627615063 % done
66.94560669456067 % done
71.1297071129707 % done
75.31380753138075 % done
79.4979079497908 % done
83.68200836820084 % done
87.86610878661088 % done
92.05020920502092 % done
100.418410041841 % done
cleaning pvt_manipur.csv
getting rid of duplicates
got rid of duplicates
0.0 % done
4.166666666666667 % done
8.333333333333334 % done
12.5 % done
16.666666666666668 % done
20.833333333333332 % done
25.0 % done
29.166666666666668 % done
33.333333333333336 % done
37.5 % done
41.666666666666664 % done
45.833333333333336 %